# Etape 1: Débuter avec Scapy
Dans cette première étape, nous importons le module Scapy et affichons différents paramètres réseau, comme la table de routage ou la passerelle par défaut.

In [ ]:
from scapy.all import *
print(f"La version de Scapy est {conf.version}.")
print(f"\nL'interface par défaut utilisée pour l'émission et la réception des paquets est {conf.iface}.")
print(f"\nLa table de routage utilisée est : \n {conf.route}.")
print('\nLa passerelle par défaut est :', conf.route.route("0.0.0.0")[2])

Nous commençons par créer un paquet nommé `paquet1`, qui a pour source et destination l'adresse IP `127.0.0.1`. Ce paquet contient un datagramme UDP vide.

In [2]:
paquet1=IP()/UDP()
paquet1.show()

###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = udp
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \
###[ UDP ]###
     sport     = domain
     dport     = domain
     len       = None
     chksum    = None



Ensuite, nous créons une trame Ethernet nommée `frm`, dont l'adresse MAC source est `11:22:33:44:55:66` et l'adresse MAC de destination est `00:0A:1F:3B:4E:64`. Cette trame encapsule un paquet IP envoyé de `192.168.1.1` à l'adresse IP de `www.univ-pau.fr`. Scapy effectue automatiquement une requête DNS pour obtenir l'adresse IP correspondante.  
Ce paquet IP contient un segment TCP avec le drapeau `SYN` activé (indiquant une tentative d'établissement de connexion), à destination du port 80 (HTTP). Le paquet contient également une charge utile (payload) en texte brut : "`C'est vraiment bien Scapy`"

In [3]:
MACs='11:22:33:44:55:66'
MACd='00:0A:1F:3B:4E:64'
IPs='192.168.1.1'
IPd='www.univ-pau.fr'
frm=Ether(src=MACs, dst=MACd)/IP(src=IPs, dst=IPd)/TCP(flags='SA')/"C'est vraiment bien Scapy"
frm.show()
frm.show2()

###[ Ethernet ]###
  dst       = 00:0A:1F:3B:4E:64
  src       = 11:22:33:44:55:66
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = None
     tos       = 0x0
     len       = None
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = None
     src       = 192.168.1.1
     dst       = Net("www.univ-pau.fr/32")
     \options   \
###[ TCP ]###
        sport     = ftp_data
        dport     = http
        seq       = 0
        ack       = 0
        dataofs   = None
        reserved  = 0
        flags     = SA
        window    = 8192
        chksum    = None
        urgptr    = 0
        options   = []
###[ Raw ]###
           load      = b"C'est vraiment bien Scapy"

###[ Ethernet ]###
  dst       = 00:0a:1f:3b:4e:64
  src       = 11:22:33:44:55:66
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 65
     id        = 1
     flags     = 
     frag      = 

## Lecture d’un fichier PCAPNG et analyse des en-têtes
Nous chargeons le fichier *Ping_Google.pcapng*, situé dans le dossier Capture. Cette capture contient 4 paquets.

In [14]:
trames=rdpcap("Capture/Ping_Google.pcapng")
print("La capture comprend les paquets suivants :\n")
trames.summary()

La capture comprend les paquets suivants :

Ether / IP / ICMP 192.168.1.48 > 8.8.8.8 echo-request 0 / Raw
Ether / IP / ICMP 8.8.8.8 > 192.168.1.48 echo-reply 0 / Raw
Ether / IP / ICMP 192.168.1.48 > 8.8.8.8 echo-request 0 / Raw
Ether / IP / ICMP 8.8.8.8 > 192.168.1.48 echo-reply 0 / Raw


In [15]:
print("contenue de la 1er trame:")
trames[0]

contenue de la 1er trame:


<Ether  dst=f4:ca:e5:5d:b5:c8 src=8c:85:90:c5:71:56 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=84 id=3173 flags= frag=0 ttl=64 proto=icmp chksum=0x9c5c src=192.168.1.48 dst=8.8.8.8 |<ICMP  type=echo-request code=0 chksum=0xf4cc id=0x5662 seq=0x0 unused=b'' |<Raw  load=b'a\xc0\xbd\xd2\x00\x05\xa25\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567' |>>>>

In [17]:
print(f"MAC source : {trames[0][Ether].src}")
print(f"MAC destination : {trames[0][Ether].dst}")
print(f"ethertype : {trames[0][Ether].type}")
print(f"IP source : {trames[0][IP].src}")
print(f"IP destination : {trames[0][IP].dst}")
print(f"protocole id : {trames[0][IP].proto}")
print(f"ICMP type id : {trames[0][ICMP].type}")
print(f"ICMP data : {trames[0][Raw].load.split(sep=None)[2]}")

MAC source : 8c:85:90:c5:71:56
MAC destination : f4:ca:e5:5d:b5:c8
ethertype : 2048
IP source : 192.168.1.48
IP destination : 8.8.8.8
protocole id : 1
ICMP type id : 8
ICMP data : b'!"#$%&\'()*+,-./01234567'


## Requêtte ICMP (ping)
Nous construisons ici une requête ICMP de type echo request à destination de l'adresse IP `10.2.12.4` (un serveur sur le reseau de l'IUT).  
Cette requête inclut le message texte suivant dans sa charge utile : "`coucou`"  
Le paquet est ensuite envoyé via l’interface réseau.

In [45]:
ping=IP(dst='10.2.12.4')/ICMP()/"coucou"
ok, nonok = srloop(ping, count=3, inter=1) # envoie 3 paquets et attend une reponse
ok[0] # lecture du 1er paquet reçut

RECV 1: IP / ICMP 10.2.12.4 > 10.8.32.245 echo-reply 0 / Raw / Padding
RECV 1:  IP / ICMP 10.2.12.4 > 10.8.32.245 echo-reply 0 / Raw / Padding
RECV 1:  IP / ICMP 10.2.12.4 > 10.8.32.245 echo-reply 0 / Raw / Padding
        
Sent 3 packets, received 3 packets. 100.0% hits.


QueryAnswer(query=<IP  frag=0 proto=icmp dst=10.2.12.4 |<ICMP  |<Raw  load=b'coucou' |>>>, answer=<IP  version=4 ihl=5 tos=0x0 len=34 id=52473 flags= frag=0 ttl=62 proto=icmp chksum=0x6edf src=10.2.12.4 dst=10.8.32.245 |<ICMP  type=echo-reply code=0 chksum=0xb7b7 id=0x0 seq=0x0 unused=b'' |<Raw  load=b'coucou' |<Padding  load=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' |>>>>)

## Sniffing de paquets
Nous utilisons la fonction `sniff()` pour capturer le trafic réseau, en appliquant un filtre spécifique aux paquets ICMP.  
Une fonction personnalisée `print_icmp` est définie pour afficher les informations pertinentes de chaque paquet ICMP capturé, comme l'adresse source, l'adresse de destination, le type de message ICMP, etc.

In [ ]:
#from scapy.all import *
ICMP_types={ 0 : 'Echo-Reply', 3 : 'Destination Unreachable', 8 : 'Echo'}
def print_icmp (packet) :
    type=packet[ICMP].type
    ips=packet[IP].src
    ipd=packet[IP].dst
    if ips==iface_ip : # on regarde si l'IP source corespont ou non a notre interface pour determinée si c'est un paquet envoyer ou reçus.
        print(f"Emission d'un paquet ICMP {ICMP_types[type]} vers {ipd}")
    else :
        print(f"Réception d'un paquet ICMP {ICMP_types[type]} en provenance de {ips}")
iface_ip=get_if_addr(conf.iface)
sniff(filter="icmp", prn=print_icmp, store=0, iface="Wi-Fi", count=4) # on recupére 4 paquets ICMP de l'interface de reseau du PC, pour chaque paquets, on apelle la fonction print_icmp

Emission d'un paquet ICMP Echo vers 10.2.12.1
Réception d'un paquet ICMP Echo-Reply en provenance de 10.2.12.1
Emission d'un paquet ICMP Echo vers 10.2.12.1
Réception d'un paquet ICMP Echo-Reply en provenance de 10.2.12.1


<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>